<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/merge_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*This notebook is based on the notebook from [Python Data Science Handbook by Jake VanderPlas*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). 

# Combining Datasets: Merge and Join

In [2]:
import pandas as pd
import numpy as np

### One-to-one joins

In [16]:
df1 = pd.DataFrame({'Empleado': ['Juan', 'Jose', 'Luis', 'Santiago'],
                    'grupo': ['Contabilidad', 'Ingeniería', 'Ingeniería', 'Recursos Humanos']})
df2 = pd.DataFrame({'Empleado': ['Luis', 'Juan', 'Jose', 'Santiago'],
                    'anio_ingreso': [2004, 2008, 2012, 2014]})

print("df1 \n",df1)

print(" ")

print("df2 \n",df2)


df1 
    Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos
 
df2 
    Empleado  anio_ingreso
0      Luis          2004
1      Juan          2008
2      Jose          2012
3  Santiago          2014


Unir un solo dataframe

In [17]:
df3 = pd.merge(df1, df2)
df3

,Empleado,grupo,anio_ingreso
0,Juan,Contabilidad,2008
1,Jose,Ingeniería,2012
2,Luis,Ingeniería,2004
3,Santiago,Recursos Humanos,2014


Se detecta la columna empleado y se une por eso, se altera el orden, se eliminan los indices.

### Many-to-one joins

# Many-to-one joins

Se preservan los duplicados


In [23]:
df4 = pd.DataFrame({'grupo': ['Contabilidad', 'Ingeniería', 'Recursos humanos'],
                    'Supervisor': ['Carlos', 'Guido', 'Esteban']})
pd.merge(df3, df4)

,Empleado,grupo,anio_ingreso,Supervisor
0,Juan,Contabilidad,2008,Carlos
1,Jose,Ingeniería,2012,Guido
2,Luis,Ingeniería,2004,Guido


Se repite la información del supervisor

### Many-to-many joins

Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [24]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """
    
{0}{1}
    """
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)



df5 = pd.DataFrame({'grupo': ['Contabilidad', 'Contabilidad',
                              'Ingeniería', 'Ingeniería', 'Recursos Humanos', 'Recursos Humanos'],
                    'habilidades': ['matemática', 'excel', 'codificación', 'linux',
                               'excel', 'organización']})
display('df1', 'df5', "pd.merge(df1, df5)")

,Empleado,grupo
0,Juan,Contabilidad
1,Jose,Ingeniería
2,Luis,Ingeniería
3,Santiago,Recursos Humanos
,grupo,habilidades
0,Contabilidad,matemática
1,Contabilidad,excel
2,Ingeniería,codificación
3,Ingeniería,linux
4,Recursos Humanos,excel


## Specification of the Merge Key

### The ``on`` keyword
``on`` usa el nombre de la columna

In [27]:
display('df1', 'df2', "pd.merge(df1, df2, on='Empleado')")

df1
   Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos

df2
   Empleado  anio_ingreso
0      Luis          2004
1      Juan          2008
2      Jose          2012
3  Santiago          2014

pd.merge(df1, df2, on='Empleado')
   Empleado             grupo  anio_ingreso
0      Juan      Contabilidad          2008
1      Jose        Ingeniería          2012
2      Luis        Ingeniería          2004
3  Santiago  Recursos Humanos          2014

Esto funciona si ambas tienen el mismo nombre

### The ``left_on`` and ``right_on`` keywords

Cuando las columnas no se llaman igual

In [28]:
df3 = pd.DataFrame({'nombre': ['Juan', 'Jose', 'Luis', 'Santiago'],
                    'Salario': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="Empleado", right_on="nombre")')

df1
   Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos

df3
     nombre  Salario
0      Juan    70000
1      Jose    80000
2      Luis   120000
3  Santiago    90000

pd.merge(df1, df3, left_on="Empleado", right_on="nombre")
   Empleado             grupo    nombre  Salario
0      Juan      Contabilidad      Juan    70000
1      Jose        Ingeniería      Jose    80000
2      Luis        Ingeniería      Luis   120000
3  Santiago  Recursos Humanos  Santiago    90000

Se puede borrar la columna extra

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [30]:
df1a = df1.set_index('Empleado')
df2a = df2.set_index('Empleado')
display('df1a', 'df2a')

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


 ``left_index`` y ``right_index`` en  ``pd.merge()``:

In [31]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


 ``DataFrame`` implementa el ``join()`` que hace un  ``merge`` con los indices:

In [32]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


Se pueden combinar indices y columnas

In [33]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='nombre')")

df1a
                     grupo
Empleado                  
Juan          Contabilidad
Jose            Ingeniería
Luis            Ingeniería
Santiago  Recursos Humanos

df3
     nombre  Salario
0      Juan    70000
1      Jose    80000
2      Luis   120000
3  Santiago    90000

pd.merge(df1a, df3, left_index=True, right_on='nombre')
              grupo    nombre  Salario
0      Contabilidad      Juan    70000
1        Ingeniería      Jose    80000
2        Ingeniería      Luis   120000
3  Recursos Humanos  Santiago    90000

El detalle del merge se puede ver en la documentación de panda (http://pandas.pydata.org/pandas-docs/stable/merging.html) .

In [36]:
df6 = pd.DataFrame({'nombre': ['Pedro', 'Pablo', 'Maria'],
                    'comida': ['pescado', 'frijo', 'pan']},
                   columns=['nombre', 'comida'])
df7 = pd.DataFrame({'nombre': ['Maria', 'Jose'],
                    'bebida': ['vino', 'cerveza']},
                   columns=['nombre', 'bebida'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7)
  nombre comida bebida
0  Maria    pan   vino

Sólo tienen en común a María

In [37]:
pd.merge(df6, df7, how='inner')

,nombre,comida,bebida
0,Maria,pan,vino


Other options for the ``how`` = ``'outer'``, ``'left'``, and ``'right'``. Outter toma todo los datos y llena con nulos los datos vacios

In [38]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='outer')
  nombre   comida   bebida
0  Pedro  pescado      NaN
1  Pablo    frijo      NaN
2  Maria      pan     vino
3   Jose      NaN  cerveza

The *left join* and *right join* 

In [39]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='left')
  nombre   comida bebida
0  Pedro  pescado    NaN
1  Pablo    frijo    NaN
2  Maria      pan   vino

In [40]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='right')
  nombre comida   bebida
0  Maria    pan     vino
1   Jose    NaN  cerveza

Conflicto en el nombre de las columnas


In [48]:
df8 = pd.DataFrame({'nombre': ['Alberto', 'Juan', 'Luis', 'Santiago'],
                    'ranking': [1, 2, 3, 4]})
df9 = pd.DataFrame({'nombre': ['Alberto', 'Juan', 'Luis', 'Santiago'],
                    'ranking': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on=["nombre", "ranking"])')

,nombre,ranking
0,Alberto,1
1,Juan,2
2,Luis,3
3,Santiago,4
,nombre,ranking
0,Alberto,3
1,Juan,1
2,Luis,4
3,Santiago,2
,nombre,ranking


In [49]:
display('df8', 'df9', 'pd.merge(df8, df9, on=["nombre"])')

df8
     nombre  ranking
0   Alberto        1
1      Juan        2
2      Luis        3
3  Santiago        4

df9
     nombre  ranking
0   Alberto        3
1      Juan        1
2      Luis        4
3  Santiago        2

pd.merge(df8, df9, on=["nombre"])
     nombre  ranking_x  ranking_y
0   Alberto          1          3
1      Juan          2          1
2      Luis          3          4
3  Santiago          4          2

In [50]:
display('df8', 'df9', 'pd.merge(df8, df9, on="nombre", suffixes=["_L", "_R"])')

df8
     nombre  ranking
0   Alberto        1
1      Juan        2
2      Luis        3
3  Santiago        4

df9
     nombre  ranking
0   Alberto        3
1      Juan        1
2      Luis        4
3  Santiago        2

pd.merge(df8, df9, on="nombre", suffixes=["_L", "_R"])
     nombre  ranking_L  ranking_R
0   Alberto          1          3
1      Juan          2          1
2      Luis          3          4
3  Santiago          4          2

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [53]:
# Following are shell commands to download the data
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [56]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Ranking de los estados de USA  por su densidad de población en  2010
1.  many-to-one merge nombre + población 
2. merge on ``state/region``  de  ``pop``,y ``abbreviation`` de ``abbrevs``.
``how='outer'`` para tener todos los datos

In [58]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


In [59]:
merged = merged.drop('abbreviation', 1) # drop duplicate info
merged.head()

<ipython-input-59-ffe4f2894b5d>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  merged = merged.drop('abbreviation', 1) # drop duplicate info


,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Verificamos nulos
---

looking for rows with nulls:

In [60]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [61]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Los valores nulos en la población son de puerto rico antes del 2000

Hay valores en  ``state`` que aparecen nulos ``abbrevs`` 

In [76]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array([], dtype=object)

Estos valores para PR y USA no aparecen en las abreviaturas


In [77]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

ahora hacemos merge con areas

In [78]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


se verifican los nulos



In [79]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

se verifica ``area``

In [80]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

El area de todo el pais no es relevante para el estudio

In [83]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2000, and the total population.
We'll use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed; see [High-Performance Pandas: ``eval()`` and ``query()``](03.12-Performance-Eval-and-Query.ipynb)):

In [84]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

In [85]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [86]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

In [87]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.
